# Description

In this exercise, you will need to construct a function that communicates with a locally running server.  Here we want you only to use only `telnetlib` rather than higher-level client libraries used in later lessons.  In particular, you will connect to port 2551 on `localhost`, and to the resource path `/json`.  As the name suggests, this path will return JSON responses; however, your function should return a Python dictionary derived from that. 

A trick to this exercise is that the server path does not accept GET requests, only POST or PUT.

Two more hints will help you out.  The Python function `urllib.parse.urlencode` may be used to encode Python dictionaries.  Also helpful is knowing the `Content-Type` header value of `application/x-www-form-urlencoded` which will be relevant to completing this.

A correct implementation of the function will behave like this:

```python
>>> get_response('David', 'Instructor')
{'Server': '77851768', 'name': 'David', 'title': 'Instructor'}
```

# Setup

In [ ]:
from telnetlib import Telnet
import json
from urllib.parse import urlencode

from exercise_server import start
start()

def get_response(name, title):
    # Telnet(... something ...)
    path = '/json'
    return {"Server": "Test Server"}

# Solution

In [ ]:
def get_response(name, title):
    eot = '⌁'.encode() 
    with Telnet('localhost', 2551) as conn:
        conn.interact()
        query = urlencode({'name': name, 'title': title})
        qlen = len(query)
        req = ('PUT /json HTTP/1.1\r\n'
               'Content-Type: application/x-www-form-urlencoded\r\n'
               f'Content-Length: {qlen}\r\n'
               f'\r\n'
               f'{query}').encode()
        conn.write(req)
        resp = conn.read_all().decode()
        header, body = resp.split('\r\n\r\n')
        return json.loads(body)

get_response('David', 'Instructor')

# Test Cases

In [ ]:
from itertools import product
import requests

def test_responses():
    names = ['Alice', 'Bob', 'Charlie']
    titles = ['Doctor', 'Lawyer', 'Baker']
    for name, title in product(names, titles):
        resp = requests.put('http://localhost:2551/json', data=dict(name=name, title=title))
        correct = json.loads(resp.text)
        maybe = get_response(name, title)
        assert maybe == correct, f'{maybe} / {correct}'
    
test_responses()